In [1]:
import langchain

In [2]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from dotenv import load_dotenv
import os

In [3]:
import sqlite3

connection = sqlite3.connect("example_new.db")
cursor = connection.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS inventory (
    id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    stock_quantity INTEGER NOT NULL
)
""")

cursor.executemany("""
INSERT INTO inventory (product_name, stock_quantity) VALUES (?, ?)
""", [
    ("Product A", 150),
    ("Product B", 200),
    ("Product C", 300),
    ("Product D", 100),
    ("Product E", 50)
])

connection.commit()
connection.close()

print("Database created and populated with sample data.")


Database created and populated with sample data.


In [4]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit 

In [ ]:
!pip install google-generativeai langchain-google-genai

  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
   ---------------------------------------- 0.0/175.4 kB ? eta -:--:--
   ---------------------------------------- 175.4/175.4 kB 5.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ------------------------- -------------- 0.8/1.3 MB 27.1 MB/s eta 0:00:01
   ---------------------------------------  1.3/1.3 MB 20.8 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 14.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/41.7 kB ? eta -:--:--
   ---------------------------------------- 41.7/41.7 kB ? eta 0:00:00
   ---------------------------------------- 0.0/160.1 kB ? eta -:--:--
   ---------------------------------------- 160.1/160.1 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/210.8 kB ? eta -:--:--
   ---------------------------------------- 210.8/210.8 kB 6.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.2 which is incompatible.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.3 which is incompatible.


In [10]:
from langchain.agents import initialize_agent, Tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from dotenv import load_dotenv
import os
import google.generativeai as genai

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found in environment variables")

genai.configure(api_key=GOOGLE_API_KEY)

llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.5
)

db = SQLDatabase.from_uri("sqlite:///example_new.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)
 
query1 = "How many units of Product A are in stock?"
response1 = agent.run(query1)
print("Response:", response1)



> Entering new AgentExecutor chain...
I need to query the database to find out how many units of Product A are in stock.
Action: sql_db_query
Action Input: SELECT SUM(quantity) FROM inventory WHERE product_id = 'A';
Observation: Error: (sqlite3.OperationalError) no such column: quantity
[SQL: SELECT SUM(quantity) FROM inventory WHERE product_id = 'A';]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Thought: I need to check the schema of the inventory table to see what the correct column name is.
Action: sql_db_schema
Action Input: inventory
Observation: 
CREATE TABLE inventory (
	id INTEGER, 
	product_name TEXT NOT NULL, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from inventory table:
id	product_name	stock_quantity
1	Product A	150
2	Product B	200
3	Product C	300
*/
Thought:The correct column name is stock_quantity.
Action: sql_db_query
Action Input: SELECT SUM(stock_quantity) FROM inventory WHERE product_id = 'A';
Observation: Error: (sqlite3.Ope